In [5]:
%pip install plotly scikit-learn numpy pandas

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.3/11.3 MB 9.1 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 505.5/505.5 kB 8.5 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 345.4/345.4 kB 7.5 MB/s eta 0:00:00a 0:00:01
Note: you may need to restart the kernel to use updated packages.


In [2]:
from sklearn.datasets import make_blobs
import numpy as np
import json
from collections import defaultdict, namedtuple
import plotly.express as px

#plotly.offline.init_notebook_mode()

In [24]:
rs = np.random.RandomState(42)
num_cluster = 10
num_dim = 3

centres = [[rs.rand() for _ in range(num_dim)] for _ in range(num_cluster)]
sigmas = [rs.rand() for _ in range(10)]
blobs = make_blobs(
                   n_samples=100, 
                   n_features=num_dim,
                   centers=centres,
                   cluster_std=sigmas,
                   random_state=rs
                   ) 

In [76]:
point_clouds = [[] for _ in range(num_cluster)]
for sample, cluster in zip(blobs[0], blobs[1]):
    point_clouds[int(cluster)].append(sample.tolist())
inputs = [{"vertices" : points} for points in point_clouds]
with open("synthetic.json", "wt") as out_file:
    json.dump(inputs, out_file, indent=2)

In [77]:
with open("synthetic_out.json", "rt") as in_file:
    polys = json.load(in_file)

In [78]:
print(len(polys[5]["essential_indices"]))

8


In [86]:
poly_ind = 0
vertices = np.array(polys[poly_ind]["vertices"])
constant = np.ones([len(vertices), 1], dtype=float)
vertices = np.concatenate([constant, vertices], axis=1)
certs = {extreme: np.array(cert) for extreme, cert in polys[poly_ind]["essential_certs"].items()}

for vertex, cert in certs.items():
    scores = vertices @ cert
    print(scores, vertex, np.argmin(scores))

[ 3.69357486e+00 -2.59436916e-12 -3.15791837e-12  7.49136587e+00
 -1.00000000e+00  1.88345206e+00  7.25269019e+00 -2.61857203e-12
  3.95700769e+00  2.51273734e+00] 4 4
[ 2.08959101e+00 -3.63087338e-12  1.63927999e+00  2.65557562e+01
 -2.54285482e-12  7.55564118e+00  3.76643191e+01 -1.00000000e+00
  1.26316542e+01 -1.00008890e-12] 7 7
[-2.82474044e-12  2.28341943e+00  1.42526225e-01 -1.30728761e-12
  1.54672277e+00  1.10097398e+00  2.57949698e+00  9.92488194e-01
 -9.99950123e-13 -1.00000000e+00] 9 9
[ 2.30012395e+00  1.78923630e+00 -1.00000000e+00  4.04672882e-02
 -1.85385041e-12  4.56830358e-01 -2.46469511e-12  5.89222875e-01
 -1.24700250e-12  5.48354881e-01] 2 2
[-1.00000000e+00 -1.15962795e-12  1.33384026e+00 -2.48795429e-12
  1.04660325e+00  4.38850256e-01  4.46660383e-01  5.16433924e-01
  3.49783275e-01 -1.97919459e-12] 0 0
[ 3.17766005e+00  6.26235497e+00 -1.23190347e-12 -1.00000000e+00
  3.28825501e+00  2.52203587e+00  2.03403190e+00  3.13546736e+00
 -1.51478829e-12 -1.13242749e-

In [117]:
poly_ind = 6
poly = polys[poly_ind]

dims = ["x", "y", "z"]
vertices_by_col = {dims[i]: list(col) for i, col in enumerate(zip(*poly["vertices"]))}
vertices_by_col["extreme"] = [str(i in poly["essential_indices"]) for i in range(len(poly["vertices"]))]
vertices_by_col["text"] = [str(i) for i in range(len(vertices_by_col["x"]))]

# Plot the edges

lines = {}
for start_vertex, adjacent in poly["adjacency"].items():
    for end_vertex in adjacent:
        line = frozenset((int(start_vertex), int(end_vertex)))
        count = lines.get(line, 0)
        lines[line] = count + 1
        
for line, count in lines.items():
    #print(line, count)
    assert count == 2, "Should be dupes"

lines_by_col = {dim: [] for dim in dims}
lines_by_col["line"] = []
for line in lines:
    for vertex in line:
        for dim in dims:
            lines_by_col[dim].append(vertices_by_col[dim][vertex])
        lines_by_col["line"].append(str(set(line)))

#(df=vertices_by_col, x="x", y="y", z="z", color="extreme", text="text")
fig = px.line_3d(lines_by_col, x="x", y="y", z="z", color="line")
#fig.add_scatter3d(**{dim: vertices_by_col[dim] for dim in dims})
fig.show()

In [3]:
with open("states.out") as states_file:
    states_data = json.load(states_file)

RSOut = namedtuple("RSOut", ["param", "decomp"])

states  = [RSOut(item["param"]["data"], item["minkowski_decomp"]) for item in states_data]

for state in states:
    print(state)


RSOut(param=[0.5231149599570413, 0.5704195644135327, 0.6332237039178327], decomp=[5, 8, 5, 4, 6, 0, 2, 4, 3, 3])
RSOut(param=[0.5777233325376563, 0.6594217146395991, 0.4810392429972379], decomp=[5, 8, 5, 4, 7, 0, 2, 4, 3, 3])
RSOut(param=[0.5231149599570413, 0.5704195644135327, 0.6332237039178327], decomp=[5, 8, 5, 4, 6, 0, 2, 4, 3, 3])
RSOut(param=[0.5777233325376563, 0.6594217146395991, 0.4810392429972379], decomp=[5, 8, 5, 4, 7, 0, 2, 4, 3, 3])
RSOut(param=[0.5231149599570413, 0.5704195644135327, 0.6332237039178327], decomp=[5, 8, 5, 4, 6, 0, 2, 4, 3, 3])
RSOut(param=[0.5777233325376563, 0.6594217146395991, 0.4810392429972379], decomp=[5, 8, 5, 4, 7, 0, 2, 4, 3, 3])
RSOut(param=[0.5231149599570413, 0.5704195644135327, 0.6332237039178327], decomp=[5, 8, 5, 4, 6, 0, 2, 4, 3, 3])
RSOut(param=[0.5777233325376563, 0.6594217146395991, 0.4810392429972379], decomp=[5, 8, 5, 4, 7, 0, 2, 4, 3, 3])
RSOut(param=[0.5231149599570413, 0.5704195644135327, 0.6332237039178327], decomp=[5, 8, 5, 4, 6,